Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training/using-environments/using-environments.png)

# Using environments


## Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Create environment](#Create-environment)
    1. Add Python packages
    1. Specify environment variables
1. [Submit run using environment](#Submit-run-using-environment)
1. [Register environment](#Register-environment)
1. [List and get existing environments](#List-and-get-existing-environments)
1. [Other ways to create environments](#Other-ways-to-create-environments)
    1. From existing Conda environment
    1. From Conda or pip files
1. [Docker settings](#Docker-settings)
1. [Spark and Azure Databricks settings](#Spark-and-Azure-Databricks-settings)
1. [Next steps](#Next-steps)

## Introduction

Azure ML environments are an encapsulation of the environment where your machine learning training happens. They define Python packages, environment variables, Docker settings and other attributes in declarative fashion. Environments are versioned: you can update them and retrieve old versions to revist and review your work.

Environments allow you to:
* Encapsulate dependencies of your training process, such as Python packages and their versions.
* Reproduce the Python environment on your local computer in a remote run on VM or ML Compute cluster
* Reproduce your experimentation environment in production setting.
* Revisit and audit the environment in which an existing model was trained.

Environment, compute target and training script together form run configuration: the full specification of training run.

## Setup

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration notebook](../../../configuration.ipynb) first if you haven't.

First, let's validate Azure ML SDK version and connect to workspace.

In [ ]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

## Create environment

You can create an environment by instantiating ```Environment``` object and then setting its attributes: set of Python packages, environment variables and others.

### Add Python packages

The recommended way is to specify Conda packages, as they typically come with complete set of pre-built binaries.

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")

You can also add pip packages, and specify the version of package

In [ ]:
conda_dep.add_pip_package("pillow==5.4.1")
myenv.python.conda_dependencies=conda_dep

### Specify environment variables

You can add environment variables to your environment. These then become available using ```os.environ.get``` in your training script.

In [ ]:
myenv.environment_variables = {"MESSAGE":"Hello from Azure Machine Learning"}

## Submit run using environment

When you submit a run, you can specify which environment to use. 

On the first run in given environment, Azure ML spends some time building the environment. On the subsequent runs, Azure ML keeps track of changes and uses the existing environment, resulting in faster run completion.

In [ ]:
from azureml.core import ScriptRunConfig, Experiment

myexp = Experiment(workspace=ws, name = "environment-example")

To submit a run, create a run configuration that combines the script file and environment, and pass it to ```Experiment.submit```. In this example, the script is submitted to local computer, but you can specify other compute targets such as remote clusters as well.

In [ ]:
runconfig = ScriptRunConfig(source_directory=".", script="example.py")
runconfig.run_config.target = "local"
runconfig.run_config.environment = myenv
run = myexp.submit(config=runconfig)

run.wait_for_completion(show_output=True)

## Register environment

You can manage environments by registering them. This allows you to track their versions, and reuse them in future runs. For example, once you've constructed an environment that meets your requirements, you can register it and use it in other experiments so as to standardize your workflow.

If you register the environment with same name, the version number is increased by one. Note that Azure ML keeps track of differences between the version, so if you re-register an identical version, the version number is not increased.

In [ ]:
myenv.register(workspace=ws)

## List and get existing environments

Your workspace contains a dictionary of registered environments. You can then use ```Environment.get``` to retrieve a specific environment with specific version.

In [ ]:
for name,env in ws.environments.items():
    print("Name {} \t version {}".format(name,env.version))

restored_environment = Environment.get(workspace=ws,name="myenv",version="1")

print("Attributes of restored environment")
restored_environment

## Other ways to create environments

### From existing Conda environment

You can create an environment from existing conda environment. This make it easy to reuse your local interactive environment in Azure ML remote runs. For example, if you've created conda environment using
```
conda create -n mycondaenv
```
you can create Azure ML environment out of that conda environment using
```
myenv = Environment.from_existing_conda_environment(name="myenv",conda_environment_name="mycondaenv")
```

### From conda or pip files

You can create environments from conda specification or pip requirements files using
```
myenv = Environment.from_conda_specification(name="myenv", file_path="path-to-conda-specification-file")

myenv = Environment.from_pip_requirements(name="myenv", file_path="path-to-pip-requirements-file")
```


## Docker settings

Docker container provides an efficient way to encapsulate the dependencies. When you enable Docker, Azure ML builds a Docker image and creates a Python environment within that container, given your specifications. The Docker images are reused: the first run in a new environment typically takes longer as the image is build.

**Note:** For runs on local computer or attached virtual machine, that computer must have Docker installed and enabled. Machine Learning Compute has Docker pre-installed.

Attribute ```docker.enabled``` controls whether to use Docker container or host OS for execution. 

In [ ]:
myenv.docker.enabled = True

You can specify custom Docker base image and registry. This allows you to customize and control in detail the guest OS in which your training run executes. whether to use GPU, whether to use shared volumes, and shm size.

In [ ]:
myenv.docker.base_image
myenv.docker.base_image_registry

You can also specify whether to use GPU or shared volumes, and shm size.

In [ ]:
myenv.docker.gpu_support
myenv.docker.shared_volumes
myenv.docker.shm_size

## Spark and Azure Databricks settings

In addition to Python and Docker settings, Environment also contains attributes for Spark and Azure Databricks runs. These attributes become relevant when you submit runs on those compute targets.

## Next steps

Learn more about remote runs on different compute targets:

* [Train on ML Compute](../../train-on-amlcompute)

* [Train on remote VM](../../train-on-remote-vm)